In [1]:
import gzip
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [2]:
yjmob1 = 'yjmob100k-dataset1.csv.gz' # dataset under normal scenes
yjmob_df = pd.read_csv(yjmob1, compression='gzip').sort_values(by=['uid', 'd', 't'], ignore_index=True)

In [4]:
df

,uid,d,t,x,y,combined_xy
335376,271,0,12,102,107,21301
335377,271,0,13,109,95,18908
335378,271,0,14,116,90,17915
335379,271,0,15,121,101,20120
335380,271,0,16,119,93,18518
...,...,...,...,...,...,...
111130882,99568,74,35,120,87,17319
111130883,99568,74,36,129,81,16128
111130884,99568,74,42,129,81,16128
111130885,99568,74,43,140,74,14739


In [27]:
USED_DATA_SIZE = 200 # only 200 data for the testing session

In [17]:
# Retrieve all ids
uids = yjmob_df['uid'].unique()

# Select id
rand_indicies = [random.randint(0, len(uids)) for _ in range(USED_DATA_SIZE)] 
selected_uids = [uid for uid in uids[rand_indicies]]

# load data into pd df
df = yjmob_df[yjmob_df['uid'].isin(selected_uids)]

# Time
# df['combined_t'] = df['d']*47+df['t']

# Gird-Location
def spatial_token(x, y):
    return (x-1)+(y-1)*200
df['combined_xy'] = df.apply(lambda row: spatial_token(row['x'], row['y']), axis=1)

# Sort values and retrieve only the values of the useful data
df = df.sort_values(by=['uid', 't'])[['uid', 't', 'combined_xy']]

/var/folders/hx/xp23lpqx4ndfxcvp3qj_bdgr0000gn/T/ipykernel_89090/533495689.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined_xy'] = df.apply(lambda row: spatial_token(row['x'], row['y']), axis=1)


In [18]:
df.head(5)

,uid,t,combined_xy
90723,68,0,20531
90839,68,0,20531
90889,68,0,20531
90946,68,0,20531
90981,68,0,20531


your_data_neural = {
    'user1': {
        'sessions': [(loc1, tim1), (loc2, tim2), ...],
        'train': [0, 1, 2],
        'test': [3, 4]
    },
    'user2': {
        'sessions': [(loc1, tim1), (loc2, tim2), ...],
        'train': [0, 1],
        'test': [2]
    },
    ...
}

In [19]:
# Split indices into train and test
# 7-3 Train-Test split
def split_indices(n, test_ratio=0.3):
    indices = list(range(n))
    test_size = int(n * test_ratio)
    train_indices = indices[:-test_size] if test_size else indices
    test_indices = indices[-test_size:] if test_size else []
    return train_indices, test_indices

In [28]:
TRAJECTORY_DATA_SIZE = 10 # how many trajectroy data used per user

In [20]:
# Transforming DataFrame to desired format
data_neural = {}
for uid, group in df.groupby('uid'):
    # Limit to the first 10 trajectory data for each user (small sample for testing)
    # Note: in my Transformer, it is using 600 trajectory data to train the final location
    group = group.head(TRAJECTORY_DATA_SIZE)
    
    # Constructing sessions as a dictionary where key is an index and value is a tuple (location, time)
    sessions = {idx: (loc, tim) for idx, (tim, loc) in enumerate(zip(group['t'], group['combined_xy']))}
    train_indices, test_indices = split_indices(len(sessions))
    data_neural[uid] = {
        'sessions': sessions,
        'train': train_indices,
        'test': test_indices
    }

In [21]:
len(list(data_neural.items()))

200

In [22]:
list(data_neural.items())[0]

(68,
 {'sessions': {0: (20531, 0),
   1: (20531, 0),
   2: (20531, 0),
   3: (20531, 0),
   4: (20531, 0),
   5: (20531, 0),
   6: (20531, 0),
   7: (20531, 0),
   8: (20531, 0),
   9: (20531, 0)},
  'train': [0, 1, 2, 3, 4, 5, 6],
  'test': [7, 8, 9]})

In [23]:
output = {}

In [24]:
output['data_neural'] = data_neural
output['uid_list'] = selected_uids

In [25]:
# output['uid_list']

In [26]:
# Save the dictionary to a pickle file
import pickle

with open('my_data_neural.pk', 'wb') as f:
    pickle.dump(output, f)